# Sentiment analysing RNN model 

### Main objective: find the most efficient RNN-based DL model capable of performing sentiment analysis of common English sentences through various RNN and word tokenisation approaches.  